# Louvain Community Detection

The Louvain method of community detection is a greedy heirarical clsutering algorithm which seeks to optimize Modularity as it progresses. Louvain starts with each vertex in its own clusters and iteratively merges groups using graph contraction.  

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Louvain_Modularity

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score


To compute the Louvain cluster in cuGraph use: 
**nvLouvain(G)**
* __G__: A cugraph.Graph object

Returns:

* tupal __lovain dataframe__ and __modularity__


* __louvain__: cudf.DataFrame with two names columns:
    * louvain["vertex"]: The vertex id.
    * louvain["partition"]: The assigned partition.
    
* __modularity__ : the overall modularity of the graph

All vertices with the same partition ID are in the same cluster

*Blondel, V. D., Guillaume, J.-L., Lambiotte, R., and Lefebvre, E. Fast unfolding of communities in large networks. Journal of statistical mechanics: theory and experiment 2008, 10 (2008), P10008.*

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


## cuGraph 0.6 Notice 
cuGraph version 0.6 has a number of limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations

### Prep

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from collections import OrderedDict

## Read data using cuDF

In [2]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

In [3]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [4]:
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [5]:
# The algorithm requires that there are vertex weights
gdf["data"] = cudf.Series(np.zeros(len(gdf), dtype=np.float32))

In [6]:
gdf.dtypes

src        int32
dst        int32
src_0      int32
dst_0      int32
data     float32
dtype: object

In [7]:
# create a Graph 
G1 = cugraph.Graph()
G1.add_edge_list(gdf["src_0"], gdf["dst_0"], gdf["data"])

In [8]:
# Call Louvain on the graph
df, mod = cugraph.nvLouvain(G1) 

In [9]:
# Print the modularity score
print('Modularity was {}'.format(mod))
print()

Modularity was 1.0000007133930922



In [10]:
df.dtypes

vertex       int32
partition    int32
dtype: object

In [11]:
# How many patitions where found
part_ids = df["partition"].unique()

In [12]:
print("Found " + str(len(part_ids)) + " partition")

Found 1 partition


In [15]:
for i in range(len(df)):
    print("Vertex ID: " + str(df["vertex"][i]) + "  is in partition " + str(df["partition"][i])) 


Vertex ID: 0  is in partition 0
Vertex ID: 1  is in partition 0
Vertex ID: 2  is in partition 0
Vertex ID: 3  is in partition 0
Vertex ID: 4  is in partition 0
Vertex ID: 5  is in partition 0
Vertex ID: 6  is in partition 0
Vertex ID: 7  is in partition 0
Vertex ID: 8  is in partition 0
Vertex ID: 9  is in partition 0
Vertex ID: 10  is in partition 0
Vertex ID: 11  is in partition 0
Vertex ID: 12  is in partition 0
Vertex ID: 13  is in partition 0
Vertex ID: 14  is in partition 0
Vertex ID: 15  is in partition 0
Vertex ID: 16  is in partition 0
Vertex ID: 17  is in partition 0
Vertex ID: 18  is in partition 0
Vertex ID: 19  is in partition 0
Vertex ID: 20  is in partition 0
Vertex ID: 21  is in partition 0
Vertex ID: 22  is in partition 0
Vertex ID: 23  is in partition 0
Vertex ID: 24  is in partition 0
Vertex ID: 25  is in partition 0
Vertex ID: 26  is in partition 0
Vertex ID: 27  is in partition 0
Vertex ID: 28  is in partition 0
Vertex ID: 29  is in partition 0
Vertex ID: 30  is in

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___